# 52. GPU 지원

- 병렬 계산에는 GPU가 훨씬 뛰어나므로 이번 단계에서는 GPU에서 구동하기 위한 구조를 만들 것임

## 52.1 쿠파이 설치 및 사용 방법

- 쿠파이는 GPU를 활용하여 병렬 계산을 해주는 라이브러리
- $ pip install cupy  

- **$ conda install -c conda-forge cupy (위 명령어로 설치가 안 됨)**  
- DeZero에서 넘파이를 사용하는 부분을 쿠파이로 바꾸면 됨
    - ```import numpy as np```
    - ```import cupy as cp```

### 아래와 같이 넘파이 지식을 쿠파이에서도 그대로 활용 가능

In [3]:
import cupy as cp

x = cp.arange(6).reshape(2,3)
print(x)

y = x.sum(axis=1)
print(y)

[[0 1 2]
 [3 4 5]]
[ 3 12]


### 넘파이 -> 쿠파이 전환
- cp.asarray: 넘파이 -> 쿠파이  
- cp.asnumpy: 쿠파이 -> 넘파이  

**실무 딥러닝에서는 다량의 데이터를 다루므로 이러한 변환이 병목으로 작용될 수 있으니 전송 횟수를 최소로 하는 것이 중요**

In [5]:
import numpy as np
import cupy as cp

# 넘파이 -> 쿠파이
n = np.array([1,2,3])
c = cp.asarray(n)
assert type(c) == cp.ndarray

# 쿠파이 -> 넘파이
c = cp.array([1,2,3])
n = cp.asnumpy(c)
assert type(n) == np.ndarray

## 52.2 쿠다 모듈

- 쿠파이 관련 함수는 cuda.py에 모아둠

In [6]:
import numpy as np
gpu_enable = True
try:
    import cupy as cp
    cupy = cp
except ImportError:
    gpu_enable = False
from dezero import Variable

### get_array_module
- 입력 x는 Variable 또는 ndarray여야 한다.  
- `gpu_enable`가 False라면 np를 반환한다.  

### as_numpy
- 넘파이의 ndarray로 변환  

## as_cupy
- 쿠파이의 ndarray로 변환

In [7]:
def get_array_module(x):
    """Returns the array module for `x`.
    Args:
        x (dezero.Variable or numpy.ndarray or cupy.ndarray): Values to
            determine whether NumPy or CuPy should be used.
    Returns:
        module: `cupy` or `numpy` is returned based on the argument.
    """
    if isinstance(x, Variable):
        x = x.data

    if not gpu_enable:
        return np
    xp = cp.get_array_module(x)
    return xp


def as_numpy(x):
    """Convert to `numpy.ndarray`.
    Args:
        x (`numpy.ndarray` or `cupy.ndarray`): Arbitrary object that can be
            converted to `numpy.ndarray`.
    Returns:
        `numpy.ndarray`: Converted array.
    """
    if isinstance(x, Variable):
        x = x.data

    if np.isscalar(x):
        return np.array(x)
    elif isinstance(x, np.ndarray):
        return x
    return cp.asnumpy(x)


def as_cupy(x):
    """Convert to `cupy.ndarray`.
    Args:
        x (`numpy.ndarray` or `cupy.ndarray`): Arbitrary object that can be
            converted to `cupy.ndarray`.
    Returns:
        `cupy.ndarray`: Converted array.
    """
    if isinstance(x, Variable):
        x = x.data

    if not gpu_enable:
        raise Exception('CuPy cannot be loaded. Install CuPy!')
    return cp.asarray(x)

## 52.3 Variable/Layer/DataLoader 클래스 추가 구현

- DeZero의 다른 클래스들에 GPU 대응 기능을 추가함

1. \_\_init__: cupy 임포트에 성공하면 두 배열 타입을 동적으로 변경할 수 있도록 array_types를 (np.ndarray, cupy.ndarray)로 설정  
2. backward: 데이터(self.data)의 타입에 따라 넘파이 또는 쿠파이 중 하나의 다차원 배열을 생성하게 함.

In [9]:

try:
    import cupy
    array_types = (np.ndarray, cupy.ndarray)
except ImportError:
    array_types = (np.ndarray)  # (1)


class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, array_types):  # (1)
                raise TypeError('{} is not supported'.format(type(data)))

    def backward(self, retain_grad=False, create_graph=False):
        if self.grad is None:
            xp = dezero.cuda.get_array_module(self.data)  # (2)
            self.grad = Variable(xp.ones_like(self.data))

1. to_cpu: Variable 데이터를 GPU에서 CPU로 전송  
2. to_gpu: Variabel 데이터를 CPU에서 GPU로 전송

In [10]:
class Variable:
    ...
    def to_cpu(self):
        if self.data is not None:
            self.data = dezero.cuda.as_numpy(self.data)

    def to_gpu(self):
        if self.data is not None:
            self.data = dezero.cuda.as_cupy(self.data)

Layer 클래스의 매개변수를 CPU 또는 GPU에 전송하는 기능 필요

In [11]:
class Layer:
    
    def to_cpu(self):
        for param in self.params():
            param.to_cpu()

    def to_gpu(self):
        for param in self.params():
            param.to_gpu()

Line3, Lind6(gpu=False), Line 12, Line 25~27, Line 32~36 추가  
1. DataLoader 클래스는 데이터셋을 미니배치로 뽑는 역할 수행  
2. \_\_next__ 메서드에서 미니배치 생성  
3. 인스턴스 변수 중 gpu 플래그를 확인하여 쿠파이와 넘파이 중 알맞는 다차원 배열 생성

In [12]:
...
import numpy as np
from dezero import cuda

class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=True, gpu=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.data_size = len(dataset)
        self.max_iter = math.ceil(self.data_size / batch_size)
        self.gpu = gpu

        self.reset()
        
    def __next__(self):
        if self.iteration >= self.max_iter:
            self.reset()
            raise StopIteration

        i, batch_size = self.iteration, self.batch_size
        batch_index = self.index[i * batch_size:(i + 1) * batch_size]
        batch = [self.dataset[i] for i in batch_index]

        xp = cuda.cupy if self.gpu else np
        x = xp.array([example[0] for example in batch])
        t = xp.array([example[1] for example in batch])

        self.iteration += 1
        return x, t

    def to_cpu(self):
        self.gpu = False

    def to_gpu(self):
        self.gpu = True

## 52.4 함수 추가 구현

- GPU 대응과 관련하여 함수를 수정함

In [13]:
from dezero import cuda

class Sin(Function):
    def forward(self, x):
        xp = cuda.get_array_module(x) # x가 넘파이면 np.sin, 쿠파이면 cp.sin사용
        y = xp.sin(x)
        return y

    def backward(self, gy):
        x, = self.inputs
        gx = gy * cos(x)
        return gx

NameError: name 'Function' is not defined

1. as_array 함수에 새로운 인수 `array_module`을 추가  
2. `array_module`: 넘파이 또는 쿠파이 중 하나의 값을 취한다.  
3. add, mul 함수 등의 사칙연산이 새로운 as_array를 사용하도록 수정

In [ ]:
def as_array(x, array_module=np):
    if np.isscalar(x):
        return array_module.array(x)
    return x

def add(x0, x1):
    x1 = as_array(x1, dezero.cuda.get_array_module(x0.data))
    return Add()(x0, x1)

def mul(x0, x1):
    x1 = as_array(x1, dezero.cuda.get_array_module(x0.data))
    return Mul()(x0, x1)

## 52.5 GPU로 MNIST 학습하기

- MINST 학습 코드를 GPU에서 실행해 봅시다. 

#### GPU를 사용할 수 있는 환경에서 DataLoader가 모델 데이터를 GPU로 전송

In [ ]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import time
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP


max_epoch = 5
batch_size = 100

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

# GPU mode
if dezero.cuda.gpu_enable: # 추가
    train_loader.to_gpu()  # 추가
    model.to_gpu()         # 추가

for epoch in range(max_epoch):
    start = time.time()
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    elapsed_time = time.time() - start
    print('epoch: {}, loss: {:.4f}, time: {:.4f}[sec]'.format(
        epoch + 1, sum_loss / len(train_set), elapsed_time))


**Google Colab에서 실행한 결과 1에포크당 1.5초 소요**  
참고) CPU로 실행시 1에포크당 8초 소요, **GPU로 실행하면 5배 빠르다!**



<img src="image/그림52-1.png" width="50%" height="50%"></img>  

# 53. 모델 저장 및 읽어오기
- 목표  
1. 모델이 가지는 매개변수를 외부 파일로 저장  
2. 저정한 파일을 다시 읽어오는 기능 구현  
학습 중인 모델의 '스냅샷'을 저장하거나 이미 학습된 매개변수를 읽어와서 추론만 수행 가능  
### ndarray 인스턴스를 외부 파일로 저장(data가 저장되어 있음)

참고) GPU 실행 환경에서는 쿠파이 텐서를 넘파이 텐서로 변환 후 외부 파일 저장

## 53.1 넘파이의 save 함수와 load 함수
- np.save: ndarray 인스턴스를 외부 파일로 저장  
- np.load: 이미 저장되어 있는 데이터를 읽어올 때는 np.load 함수 이용

In [14]:
import numpy as np

x = np.array([1,2,3])
np.save('test.npy', x)

x = np.load('test.npy')
print(x)

[1 2 3]


In [17]:
# 여러 개도 가능
# 딕셔너리도 저장 가능

x1 = np.array([1,2,3])
x2 = np.array([4,5,6])
# data = {'x1' : x1, 'x2' : x2}

np.savez('test.npz', x1=x1, x2=x2)
# np.saves('test.npz', **data)

arrays = np.load('test.npz')
x1 = arrays['x1']
x2 = arrays['x2']
print(x1)
print(x2)

[1 2 3]
[4 5 6]


## 53.2 Layer 클래스의 매개변수를 평형하게
- 아래와 같은 계층 구조로부터 Parameter를 '하나의 평평한 딕셔너리'(중첩되지 않은 딕셔너리)로 뽑아내기  --> \_flatten_params 메서드 추가

<img src="image/그림53-1.png" width="50%" height="50%"></img>  

In [ ]:
layer = Layer()

l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))

params_dict = {}
layer._flatten_params(params_dict)
print(params_dict)
# {'p2': variable(2), 'l1/p1' : variable(1), 'p3' : variabel(3)}

1. 인수로 딕셔너리인 params_dict, 텍스트인 parent_key 받는다.  
2. 실제 객체는 obj = self.\_\_dict__[name]으로 꺼낸다.  
3. 꺼낸 obj가 Layer라면 obj의 \_flatten_params 메서드를 호출  
4. 메서드가 재귀적으로 호출되므로 모든 Parameter를 한 줄로 평탄화시켜 꺼낼 수 있다.

In [ ]:
class Layer:
    
    def _flatten_params(self, params_dict, parent_key=""):
        for name in self._params:
            obj = self.__dict__[name]
            key = parent_key + '/' + name if parent_key else name

            if isinstance(obj, Layer):
                obj._flatten_params(params_dict, key)
            else:
                params_dict[key] = obj

## 53.3 Layer 클래스의 save 함수와 load 함수
- save_weights: self.to_cpu를 호출해 데이터가 넘파이 ndarray임을 보장, ndarray 인스턴스를 값으로 갖는 딕셔너리 array_dict를 생성, np.savez_compressed 함수를 호출해 데이터를 외부 파일로 저장    
- load_weights: np.load 함수로 데이터를 읽어 들인 후 대응하는 키 데이터를 매개변수로 저장

In [ ]:
import os
class Layer:
    def save_weights(self, path):
        self.to_cpu()

        params_dict = {}
        self._flatten_params(params_dict)
        array_dict = {key: param.data for key, param in params_dict.items()
                      if param is not None}
        try:
            np.savez_compressed(path, **array_dict)
        except (Exception, KeyboardInterrupt) as e: # 키보드 인터럽트 발생시 파일 삭제
            if os.path.exists(path):
                os.remove(path)
            raise

    def load_weights(self, path):
        npz = np.load(path)
        params_dict = {}
        self._flatten_params(params_dict)
        for key, param in params_dict.items():
            param.data = npz[key]

이 코드를 처음 실행하면 모델의 매개변수를 무작위로 초기화한 상태에서 학습을 시작.  
1. 38줄은 학습된 매개변수를 저장  
2. 20~21줄에서는 파일로부터 매개변수들을 읽어온다. 이를 통해 학습한 매개변숫값이 모델에 설정됨.

In [ ]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import dezero
import dezero.functions as F
from dezero import optimizers
from dezero import DataLoader
from dezero.models import MLP


max_epoch = 3
batch_size = 100

train_set = dezero.datasets.MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)
model = MLP((1000, 10))
optimizer = optimizers.SGD().setup(model)

# 매개변수 읽기
if os.path.exists('my_mlp.npz'):
    model.load_weights('my_mlp.npz')

for epoch in range(max_epoch):
    sum_loss = 0

    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(t)

    print('epoch: {}, loss: {:.4f}'.format(
        epoch + 1, sum_loss / len(train_set)))

# 매개변수 저장하기
model.save_weights('my_mlp.npz')

# 54. 드롭아웃과 테스트 모드
- 과대적합이 일어나는 주요 원인  
1) 훈련 데이터가 적음: 데이터 확장으로 개선  
2) 모델의 표현력이 지나치게 높음: 가중치 가소, 드롭아웃, 배치 정규화 등 사용

**드롭아웃을 적용하려면 학습 시와 테스트 시의 처리 로직을 다르게 해야 한다.**  
\>> 학습 단계인지 테스트 단계인지 구별하는 구조 생성

## 54.1  드롭아웃이란
- 뉴런을 임의로 삭제(비활성화)하면서 학습하는 방법.  
- 학습 시에는 은닉층 뉴럭을 무작위로 골라 삭제. 삭제된 뉴런은 신호를 전송하지 않는다. 


<img src="image/그림54-1.png" width="50%" height="50%"></img>  

In [ ]:
# 10개의 뉴런으로 이루어진 층, 그 다음 층에서 드롭아웃 계층을 사용해 60%의 뉴런을 무작위로 삭제

import numpy as np

dropout_ratio = 0.6
x = np.ones(10)

mask = np.random.rand(10) > dropout_ratio # mask는 True 혹은 False인 배열
y = x * mask # mask에서 값이 False인 원소에 대응하는 x의 원소를 0으로 설정(삭제)

# 결과적으로 매회 평균 4개의 뉴런만 다음 층으로 전달.

**테스트 시에는 모든 뉴런을 사용하면서도 앙상블 학습처럼 동작하게끔 '흉내'내야 함.**  
모든 뉴런을 써서 출력을 계산, 그 결과를 '약화'  
약화시키는 비율은 학습 시에 살아남은 뉴런의 비율  
**아래 예시 코드를 보면 테스트 시에는 학습 시 생존한 비율(1-0.6 = 0.4)를 적용**

In [ ]:
# 학습 시
mask = np.random.rand(*x.shape) > dropout_ratio
y = x * mask

# 테스트 시
scale = 1 - dropout_ratio # 학습 시에 살아남은 뉴런의 비율 (1- 0.6 = 0.4)
y = x * scale

## 54.2 역 드롭아웃
- 일반적인 드롭아웃(다이렉트 드롭아웃)에서는 테스트 시에 스케일 맞추기를 했다면  
- 역 드롭아웃은 **학습 시에 적용**  

- 장점: 테스트 시 아무런 처리를 하지 않기 때문에 테스트 속도가 향상  
또한 dropout_ratio를 동적으로 변경 가능  

**이러한 이점 때문에 많은 딥러닝 프레임워크에서 역 드롭아웃 방식을 채택**

In [ ]:
# 학습 시
scale = 1 - dropout_ratio
mask = np.random.rand(*x.shape) > dropout_ratio
y = x * mask / scale

# 테스트 시
y = x

## 54.3 테스트 모드 추가
- 드롭아웃을 사용하려면 학습 단계인지 테스트 단계인지 구분이 필요  
1. Config 클래스에 train이라는 클래스 변수 추가(초깃값은 True)
2. test_mode 함수 추가: with문과 함께 사용하면 Config.train이 False로 전환, \_\_init__.py에 fro, dezero.core import test_mode 추가해 사용자 코드에서 사용할 수 있게 한다.

In [ ]:
class Config:
    enable_backprop = True
    train = True # 추가


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

# 테스트 모드 함수 추가
def test_mode():
    return using_config('train', False)

## 54.4 드롭아웃 구현
1. x는 Varibale 인스턴스 또는 ndarray 인스턴스.  
2. 쿠파이의 ndarray 인스턴스인 경우도 고려해 xp = cuda.get_array_module(x)에서 적절한 모듈 가져오기

In [18]:
def dropout(x, dropout_ratio=0.5):
    x = as_variable(x)
    
    if dezero.Config.train:
        xp = cuda.get_array_module(x)
        mask = xp.random.rand(*x.shape) > dropout_ratio
        scale = xp.array(1.0 - dropout_ratio).astype(x.dtype)
        y = x * mask / scale
        return y
    
    else:
        return x

In [19]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
from dezero import test_mode
import dezero.functions as F

x = np.ones(5)
print(x)

# When training
y = F.dropout(x)
print(y)

# When testing (predicting)
with test_mode():
    y = F.dropout(x)
    print(y)

[1. 1. 1. 1. 1.]
variable([0. 0. 0. 2. 0.])
variable([1. 1. 1. 1. 1.])


# 55. CNN 메커니즘(1)

## 55.1 CNN 신경망의 구조
- 합성곱층(Conv)과 풀링층(Pool)이 새롭게 등장  

<img src="image/그림55-1.png" width="50%" height="50%"></img>  
- 지금까지의 'Linear -> ReLU' 연결이 **'Conv -> ReLU -> (Pool)'**로 대체  
- 출력에 가까워지면 이전과 같은 'Linear -> ReLU' 조합이 사용됨

## 55.2 합성곱 연산
- 합성곱 연산은 입력 데이터에 대해 필터 윈도우를 일정 간격으로 이동시키면서 적용.  
참고) 필터를 커널이라고도 부른다.  


<img src="image/그림55-2.png" width="50%" height="50%"></img>  

<img src="image/그림55-3.png" width="50%" height="50%"></img>  

<img src="image/그림55-4.png" width="50%" height="50%"></img>  

위 그림을 보면 필터가 가로 새로 두 방향으로 이동한다.  
이처럼 두 개의 차원으로 움직인다고 하여 **2차원 합성곱층**이라고도 한다.  
이미지는 주로 2차원 합성곱층을 사용.

## 55.3 패딩
- 패딩: 입력 데이터 주위에 고정값(0과 같은 값)을 채운다.  
- 사용하는 이유: 출력 크기를 조절하기 위해(패딩이 없다면 출력 크기가 자꾸 줄어들 것)  


<img src="image/그림55-5.png" width="50%" height="50%"></img>  

## 55.4 스트라이드
- 필터를 적용하는 위치의 간격(보폭을 뜻함)  
아래 그림 55-6은 스트라이드가 2. 필터가 한 번에 두 원소씩 움직인다.  

<img src="image/그림55-6.png" width="50%" height="50%"></img>  

## 55.5 출력 크기 계산 방법
- 패딩을 키우면 출력도 커지고,  
- 스트라이드를 키우면 출력은 작아진다.  

In [20]:
def get_conv_outsize(input_size, kernel_size, stride, pad):
    return (input_size + pad * 2 - kernel_size) // stride + 1


H, W = 4, 4  # Input size
KH, KW = 3, 3  # Kernel size
SH, SW = 1, 1  # Kernel stride
PH, PW = 1, 1  # Padding size

OH = get_conv_outsize(H, KH, SH, PH)
OW = get_conv_outsize(W, KW, SW, PW)
print(OH, OW)


4 4


# 56. CNN 메커니즘(2)
- 이미지에는 가로/세로 방향뿐 아니라 RGB처럼 '채널 방향'으로도 데이터가 쌓여있다.  
- 3차원 데이터(3차원 텐서)를 다뤄야 한다.

## 56.1 3차원 텐서
- 주의: 입력 데이터와 필터의 '채널'수를 똑같이 맞춰야 한다.  
그림 56-1에서는 입력 데이터와 필터의 채널 수는 모두 3개이다.  
원한다면 필터의 가로, 세로 크기는 원하는 숫자로 설정 가능  
(3,3) -> (2,1) 또는 (1,2)

<img src="image/그림56-1.png" width="50%" height="50%"></img>  

## 56.2 블록으로 생각하기
- 3차원 텐서에 대한 합성곱 연산은 직육면체 블록으로 생각하면 쉽다.  
그림 56-2에서는 데이터가 (채널;Channel, 높이;Height, 너비;Width) 순서로 정렬  
필터도 마찬가지로 (C, KH, KW)로 표기  

**출력은 특징 맵(feature map)**이라고 한다.  
- 그림 56-2에서는 특징 맵이 한 장  
- 특징 맵을 여러 장 가지기 위해선 **다수의 필터(가중치)**를 사용  
- 그림 56-3을 확인하면 필터가 (OC, C, KH, KW)를 사용해서 출력을 (OC, OH, OW) 생성  

**편향 추가 가능**  
- 그림 56-3에 편향을 추가하면 그림 56-4와 같이 된다.  
- 편향을 채널당 하나의 값만 가진다. 형상 = (OC, 1, 1)  
편향을 형상이 다르기 때문에 브로드캐스트된 다음에 더해진다.

<img src="image/그림56-2.png" width="50%" height="50%"></img>  

<img src="image/그림56-3.png" width="50%" height="50%"></img>  

<img src="image/그림56-4.png" width="50%" height="50%"></img>  

## 56.3 미니배치 처리
- 미니배치: 신경망 학습에서는 여러 개의 입력 데이터를 하나의 미니배치로 묶어 처리  

**미니배치 처리를 위해서는 각 층을 흐르는 데이터를 '4차원 텐서'로 취급**  

- 그림 56-5를 보면 데이터 맨 앞에 배치를 위한 차원 추가  
(batch_size, channel, height, width)  
- 미니배치 처리에서는 이 4차원 텐서의 샘플 데이터 각각에 (독립적으로) 똑같은 합성곱 연상 수행

<img src="image/그림56-5.png" width="50%" height="50%"></img>  

## 56.4 풀링 층
- 풀링: 가로, 세로 공간을 작게 만드는 연산.  
- 그림 56-6은 2\*2 Max 풀링을 스트라이드 2로 수행하는 처리 절차를 보여줌.  

**최대 풀링: 해당 영역에서 값이 가장 큰 원소를 찾는 것**  
일반적으로 풀링 윈도우 크기와 스트라이드 크기는 같은 값으로 설정  


<img src="image/그림56-6.png" width="50%" height="50%"></img>  

### 풀링의 특징
1. 학습하는 매개변수가 없다. 풀링은 대상 영역에서 최댓값을 취하면 끝.  
2. 채널 수가 변하지 않는다. 계산이 채널마다 독립적으로 이루어지기 때문.   
<img src="image/그림56-7.png" width="50%" height="50%"></img>  
3. 미세한 위치 변화에 영향을 덜 받는다. 그림 56-8을 보면 입력 데이터가 오른쪽으로 1 원소만큼 어긋났지만 출력은 달라지지 않았다.
<img src="image/그림56-8.png" width="50%" height="50%"></img>  

# 57. conv2d 함수와 pooling 함수

- for문을 사용하지 않고 im2col이라는 편의 함수를 사용  
im2col: image to column(이미지에서 열로)

## 57.1 im2col에 의한 전개
- im2col은 입력 데이터를 **한 줄로 전개**하는 함수  
합성곱 연산 중 커널 계산에 편리하도록 입력 데이터를 펼쳐준다.  
3차원 텐서인 입력 데이터로부터 커널을 적용할 영역을 추출  
- 그림 57-1처럼 커널을 적용할 영역 꺼내기, 한 줄로 reshape -> 행렬(2차원 텐서) 변환  

<img src="image/그림57-1.png" width="50%" height="50%"></img>  
- 행렬 곱을 계산하면 행렬(2차원 텐서)이 출력,  
- 이 출력을 3차원 텐서로 변환
<img src="image/그림57-2.png" width="50%" height="50%"></img>  

## 57.2 conv2d 함수 구현
- kernel_size: 튜플로 주어지면 첫 번째 원소가 높이에, 두 번째 원소가 너비에 대응  
하나면 높이와 너비가 같다고 해석  
- stride, pad도 같은 방식으로 해석  
- to_matrix: True라면 커널을 적용할 영역을 추출한 후 '행렬'로 형상 변환(행렬 곱을 위해)  

<img src="image/표57-1.png" width="50%" height="50%"></img>

In [ ]:
import numpy as np
import dezero.functions as F

x1 = np.random.rand(1,3,7,7) # 배치 크기 = 1
col1 = F.im2col(x1, kernel_size=5, stride=1, pad=0, to_matrix=True)
print(col.shape)

x2 = np.random.rand(10,3,7,7)
kernel_size = (5,5)
stride = (1,1)
pad = (0,0)
col2 = F.im2col(x2, kernel_size, stride, pad, to_matrix=True)
print(col2.shape)

실행 결과  
(9, 75)  
(90, 75)

1. 첫 번째로 준비한 데이터 형상은 (1,3,7,7)  
배치 크기가 1, 채널 수가 3, 높이가 7, 너비가 7  

2. 두 번째는 첫 번째 예에서 배치크기만 10배.  

im2col 함수 적용하면 원소 수는 둘 다 75 = 채널수 3에 (5,5) 형상의 데이터  
첫 번째 예의 결과는 (9, 75)인 반면, 두 번째는 (90, 75)이 된다. (10배)

**im2col 함수를 이용해 합성곱 연산을 수행하는 Dezero 함수 구현**  
pair(x)라는 함수부터 구현--> x가 int라면 그대로 반환, 길이가 2인 튜플일 때 반환

In [ ]:
def pair(x):
    if isinstance(x, int):
        return (x,x)
    elif isinstance(x, tuple):
        assert len(x) == 2
        return x
    else:
        raise ValueError

### 합성곱 연산을 수행하는 conv2d_simple 함수 구현
1. col 변수에 입력 데이터  
2. 커널 Weight를 한 줄로 펼쳐 재정렬  
3. t 변수에 행렬 곱 계산(linear 함수를 사용해 편향까지 포함해 계산 수행)  
4. y 변수에 출력 크기를 reshape하고 transpose 적용  
<img src="image/그림57-3.png" width="50%" height="50%"></img>  

In [ ]:
def conv2d_simple(x, W, b=None, stride=1, pad=0):
    x, W = as_variable(x), as_variable(W)

    Weight = W
    N, C, H, W = x.shape
    OC, C, KH, KW = Weight.shape
    SH, SW = pair(stride)
    PH, PW = pair(pad)
    OH = get_conv_outsize(H, KH, SH, PH)
    OW = get_conv_outsize(W, KW, SW, PW)

    col = im2col(x, (KH, KW), stride, pad, to_matrix=True)
    Weight = Weight.reshape(OC, -1).transpose()
    t = linear(col, Weight, b)
    y = t.reshape(N, OH, OW, OC).transpose(0, 3, 1, 2)
    return y

### 역전파도 문제없이 구현 가능

In [ ]:
N, C, H, W = 1, 5, 15, 15
OC, (KH, KW) = 8, (3, 3)

x = variable(np.random.rand(N, C, H, W))
W = np.random.rand(OC, C, KH, KW)
y = F.conv2d_simple(x, W, b=None, stride=1, pad=1)
y.backward()

print(y.shape)
print(x.grad.shape)

실행 결과  
(1, 8, 15, 15)  
(1, 5, 15, 15)

## 57.3 Conv2d 계층 구현
- 함수가 아닌 '계층'으로서의 Conv2d 클래스 구현

**Layer 클래스를 상속하고, 초기화시 표 57-2의 인수들을 받는다.**  
<img src="image/표57-2.png" width="50%" height="50%"></img>  
주의할 점: in_channels의 기본값이 None라는 점.  이 값이 None라면 forward(x)에 주어지는 x의 형상으로부터 in_channels의 값을 얻고, 그 시점에 가중치 데이터를 초기화. (완전 연결 계층의 Linear 계층과 동일한 작동 방식)

In [ ]:
class Conv2d(Layer):
    def __init__(self, out_channels, kernel_size, stride=1,
                 pad=0, nobias=False, dtype=np.float32, in_channels=None):
        """Two-dimensional convolutional layer.
        Args:
            out_channels (int): Number of channels of output arrays.
            kernel_size (int or (int, int)): Size of filters.
            stride (int or (int, int)): Stride of filter applications.
            pad (int or (int, int)): Spatial padding width for input arrays.
            nobias (bool): If `True`, then this function does not use the bias.
            in_channels (int or None): Number of channels of input arrays. If
            `None`, parameter initialization will be deferred until the first
            forward data pass at which time the size will be determined.
        """
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.pad = pad
        self.dtype = dtype

        self.W = Parameter(None, name='W')
        if in_channels is not None:
            self._init_W()

        if nobias:
            self.b = None
        else:
            self.b = Parameter(np.zeros(out_channels, dtype=dtype), name='b')

    def _init_W(self, xp=np):
        C, OC = self.in_channels, self.out_channels
        KH, KW = pair(self.kernel_size)
        scale = np.sqrt(1 / (C * KH * KW))
        W_data = xp.random.randn(OC, C, KH, KW).astype(self.dtype) * scale
        self.W.data = W_data

    def forward(self, x):
        if self.W.data is None:
            self.in_channels = x.shape[1]
            xp = cuda.get_array_module(x)
            self._init_W(xp)

        y = F.conv2d(x, self.W, self.b, self.stride, self.pad)
        return y

## 57.4 pooling 함수 구현
- 풀링은 채널 방향과는 독립적이라는 점이 합성곱층과 다르다.  

그림 57-4와 같이 풀링의 적용 영역은 채널마다 독립적으로 전개  
<img src="image/그림57-4.png" width="50%" height="50%"></img>  

전개 후에는 전개된 행렬의 각 행렬로 최댓값을 구해 적절한 형상으로 바꾸면 끝.  
<img src="image/그림57-5.png" width="50%" height="50%"></img>  

**풀링 함수 구현은 세 단계**  
1. col 변수에 입력 데이터 전개  
2. y변수에 각 행의 최댓값 찾기  
3. 다시 y변수에 적절한 크기로 출력의 형상을 변환  

In [ ]:
def pooling_simple(x, kernel_size, stride=1, pad=0):
    x = as_variable(x)

    N, C, H, W = x.shape
    KH, KW = pair(kernel_size)
    PH, PW = pair(pad)
    SH, SW = pair(stride)
    OH = get_conv_outsize(H, KH, SH, PH)
    OW = get_conv_outsize(W, KW, SW, PW)

    col = im2col(x, kernel_size, stride, pad, to_matrix=True) # 1. 전개
    col = col.reshape(-1, KH * KW)
    y = col.max(axis=1) # 2. 최댓값
    y = y.reshape(N, OH, OW, C).transpose(0, 3, 1, 2) # 3. 형상 변환
    return y